# Task 3.2: Model Explainability - Credit Card Fraud Detection

## Objective
Explain the credit card fraud model using SHAP:
1. **Global explainability**: Which PCA components and features matter most?
2. **Local explainability**: Why specific transactions were flagged

## Important Note on Interpretability
The credit card dataset uses **PCA-transformed features** (V1-V28) for privacy.
- We can still identify *which components* drive predictions
- But we cannot directly interpret *what business meaning* those components have
- Only `Time` and `Amount` are directly interpretable

In [ ]:
# Standard imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import shap
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

# Project imports
from src.data.loader import load_creditcard_data
from src.explainability.shap_utils import (
    transform_for_explanation,
    create_tree_explainer,
    compute_shap_values,
    plot_shap_summary,
    plot_shap_bar,
    plot_shap_dependence,
    plot_shap_waterfall,
    get_example_cases,
)

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

shap.initjs()

RANDOM_STATE = 42

## 1. Load Model and Data

In [ ]:
# Load the credit card model
MODEL_PATH = project_root / "models" / "creditcard_random_forest.joblib"

model = joblib.load(MODEL_PATH)
print(f"Loaded model from: {MODEL_PATH}")
print(f"Pipeline steps: {list(model.named_steps.keys())}")

In [ ]:
# Load credit card data
DATA_PATH = project_root / "data" / "raw" / "creditcard.csv"

df = load_creditcard_data(DATA_PATH)
print(f"Loaded data: {df.shape}")

In [ ]:
# Define features
FEATURE_COLS = [col for col in df.columns if col != 'Class']
TARGET = 'Class'

X = df[FEATURE_COLS].copy()
y = df[TARGET].copy()

print(f"Features: {X.shape[1]}")
print(f"Target distribution: {y.value_counts().to_dict()}")

In [ ]:
# Recreate train/test split (same as Task 2)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"Test set size: {X_test.shape[0]:,}")

## 2. Prepare for SHAP Analysis

In [ ]:
# Feature names (all numeric for credit card)
feature_names = FEATURE_COLS

print(f"Features: {len(feature_names)}")
print(feature_names)

In [ ]:
# Transform test data (just scaling for creditcard pipeline)
X_test_transformed = transform_for_explanation(model, X_test)

print(f"Transformed test data shape: {X_test_transformed.shape}")

In [ ]:
# Create SHAP explainer
explainer = create_tree_explainer(model)

print(f"Explainer type: {type(explainer)}")

In [ ]:
# Compute SHAP values (sample for speed - credit card dataset is large)
SAMPLE_SIZE = 5000

print(f"Computing SHAP values for {min(SAMPLE_SIZE, len(X_test_transformed)):,} samples...")
print("(This may take a few minutes)")

shap_values = compute_shap_values(
    explainer,
    X_test_transformed,
    sample_size=SAMPLE_SIZE,
    random_state=RANDOM_STATE
)

print(f"SHAP values computed: {shap_values.values.shape}")

## 3. Global Explainability

### 3.1 SHAP Summary Plot

In [ ]:
# Summary plot
plot_shap_summary(
    shap_values,
    feature_names,
    max_display=20,
    plot_type="dot",
    title="SHAP Summary Plot - Credit Card Fraud Model",
    save_path=project_root / "reports" / "figures" / "shap_summary_creditcard.png"
)

In [ ]:
from IPython.display import Markdown, display

mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
top_indices = np.argsort(mean_abs_shap)[-5:][::-1]
top_features = [feature_names[i] for i in top_indices]

display(
    Markdown(
        f"""
### Interpretation: SHAP Summary (Credit Card)

**Top 5 most impactful features:**
1. `{top_features[0]}`
2. `{top_features[1]}`
3. `{top_features[2]}`
4. `{top_features[3]}`
5. `{top_features[4]}`

**Interpretation caveat:**
- V1-V28 are PCA components; their *direction* is interpretable (positive/negative impact) but not their *meaning*.
- If `Amount` or `Time` appear in top features, those provide actionable business insights.
"""
    )
)

### 3.2 SHAP Feature Importance

In [ ]:
# Bar plot
importance_df = plot_shap_bar(
    shap_values,
    feature_names,
    max_display=20,
    title="SHAP Feature Importance - Credit Card Fraud Model",
    save_path=project_root / "reports" / "figures" / "shap_importance_creditcard.png"
)

In [ ]:
# Show importance table
print("Top 15 Features by SHAP Importance:")
display(importance_df.head(15).round(4))

### 3.3 Dependence Plots for Interpretable Features

In [ ]:
# Dependence plot for Amount (interpretable feature)
if 'Amount' in feature_names:
    plot_shap_dependence(
        shap_values,
        'Amount',
        feature_names,
        title="SHAP Dependence: Amount",
        save_path=project_root / "reports" / "figures" / "shap_dependence_amount.png"
    )

In [ ]:
# Dependence plot for Time
if 'Time' in feature_names:
    plot_shap_dependence(
        shap_values,
        'Time',
        feature_names,
        title="SHAP Dependence: Time",
        save_path=project_root / "reports" / "figures" / "shap_dependence_time.png"
    )

In [ ]:
# Dependence plot for top PCA component
top_v = [f for f in importance_df['feature'].head(5) if f.startswith('V')]
if top_v:
    plot_shap_dependence(
        shap_values,
        top_v[0],
        feature_names,
        title=f"SHAP Dependence: {top_v[0]} (Top PCA Component)",
        save_path=project_root / "reports" / "figures" / f"shap_dependence_{top_v[0].lower()}.png"
    )

In [ ]:
from IPython.display import Markdown, display

display(
    Markdown(
        """
### Interpretation: Dependence Plots (Credit Card)

**`Amount`:**
- Shows how transaction amount affects fraud prediction.
- Often non-linear: both very small and very large amounts may be suspicious.

**`Time`:**
- Time since first transaction in the dataset.
- May show patterns related to time-of-day or elapsed session time.

**PCA Components (V1-V28):**
- Abstract patterns that differentiate fraud from non-fraud.
- The model learns which combinations of original features (now hidden) predict fraud.
"""
    )
)

## 4. Local Explainability

In [ ]:
# Get predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(f"Predictions made: {len(y_pred):,}")

In [ ]:
# Get example cases
example_cases = get_example_cases(
    y_test.values,
    y_pred,
    y_proba,
    n_examples=3
)

print("Example cases found:")
for case_type, indices in example_cases.items():
    print(f"  {case_type}: {len(indices)} examples")

In [ ]:
# Compute full SHAP values for local explanations
print("Computing SHAP values for local explanations...")
shap_values_full = explainer(X_test_transformed)
print("Done.")

### 4.1 True Positive Example

In [ ]:
if example_cases['true_positive']:
    tp_idx = example_cases['true_positive'][0]
    
    print(f"True Positive Example (index {tp_idx}):")
    print(f"  Actual: Fraud | Predicted: Fraud | Probability: {y_proba[tp_idx]:.4f}")
    print(f"  Amount: ${X_test.iloc[tp_idx]['Amount']:.2f}")
    
    plot_shap_waterfall(
        shap_values_full,
        tp_idx,
        feature_names,
        max_display=15,
        title=f"True Positive: Correctly caught fraud (prob={y_proba[tp_idx]:.3f})",
    )
else:
    print("No True Positive examples found.")

### 4.2 False Positive Example

In [ ]:
if example_cases['false_positive']:
    fp_idx = example_cases['false_positive'][0]
    
    print(f"False Positive Example (index {fp_idx}):")
    print(f"  Actual: Legitimate | Predicted: Fraud | Probability: {y_proba[fp_idx]:.4f}")
    print(f"  Amount: ${X_test.iloc[fp_idx]['Amount']:.2f}")
    
    plot_shap_waterfall(
        shap_values_full,
        fp_idx,
        feature_names,
        max_display=15,
        title=f"False Positive: Legitimate tx flagged (prob={y_proba[fp_idx]:.3f})",
    )
else:
    print("No False Positive examples found.")

### 4.3 False Negative Example

In [ ]:
if example_cases['false_negative']:
    fn_idx = example_cases['false_negative'][0]
    
    print(f"False Negative Example (index {fn_idx}):")
    print(f"  Actual: Fraud | Predicted: Legitimate | Probability: {y_proba[fn_idx]:.4f}")
    print(f"  Amount: ${X_test.iloc[fn_idx]['Amount']:.2f}")
    
    plot_shap_waterfall(
        shap_values_full,
        fn_idx,
        feature_names,
        max_display=15,
        title=f"False Negative: Missed fraud (prob={y_proba[fn_idx]:.3f})",
    )
else:
    print("No False Negative examples found.")

## 5. Summary: Task 3 (Credit Card) Complete

In [ ]:
from IPython.display import Markdown, display

top5_str = "\n".join([f"{i+1}. `{f}` (mean |SHAP| = {importance_df.iloc[i]['mean_abs_shap']:.4f})" 
                       for i, f in enumerate(importance_df.head(5)['feature'])])

display(
    Markdown(
        f"""
## Summary: Task 3 - Credit Card Fraud Model Explainability

### Top 5 Features by SHAP
{top5_str}

### Key Findings

1. **PCA components** dominate the model's decision making.
   - These are abstract combinations of original transaction attributes.
   - We cannot directly interpret their business meaning.

2. **Amount and Time** provide limited direct interpretability.
   - If Amount ranks highly, transaction value matters for fraud detection.
   - Time patterns may reflect temporal fraud behavior.

3. **Model behavior** is consistent with expectations.
   - SHAP values show clear separation for fraud vs non-fraud.
   - Local explanations reveal which components drive individual predictions.

### Comparison with E-commerce Model

| Aspect | E-commerce | Credit Card |
|--------|------------|-------------|
| Feature interpretability | High (meaningful names) | Low (PCA transformed) |
| Business actionability | High | Medium |
| SHAP explanation clarity | High | Medium |

### Files Saved
- `reports/figures/shap_summary_creditcard.png`
- `reports/figures/shap_importance_creditcard.png`
- `reports/figures/shap_dependence_*.png`

### Conclusion
Task 3 is complete. Both models have been explained using SHAP, providing:
- Global feature importance rankings
- Feature-value relationships (dependence plots)
- Individual prediction explanations (waterfall plots)
"""
    )
)